In [1]:
!pip -q install google-cloud-aiplatform google-api-python-client
!pip -q install langchain langchain_community openai tiktoken sentence-transformers==2.2.2 InstructorEmbedding faiss-cpu pypdf python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.1/366.1 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 897

In [2]:
!pip install ragas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.5/91.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-1

In [3]:
from google.colab import drive
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFDirectoryLoader

drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/Documents/"
loader = PyPDFDirectoryLoader(root_dir)
documents = loader.load()

Mounted at /content/gdrive


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
                                               chunk_size=400,
                                               chunk_overlap=0)

# Extract text from Document objects
texts = [doc.page_content for doc in documents]

# Join the extracted texts into a single string
all_text = "\n".join(texts)

# Split the combined text into chunks
chunks = text_splitter.split_text(all_text)

In [5]:
from langchain.vectorstores import FAISS
from InstructorEmbedding import INSTRUCTOR
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
import os
os.environ["OPENAI_API_KEY"] = "Your_API_Key"

openai_embeddings=OpenAIEmbeddings()
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Cola

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.40k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [6]:
db_instructEmbedd = FAISS.from_texts(chunks, instructor_embeddings)
instruct_retriever = db_instructEmbedd.as_retriever(search_kwargs={"k": 16})

In [7]:
db_openEmbedd = FAISS.from_texts(chunks, openai_embeddings)
open_retriever = db_openEmbedd.as_retriever(search_kwargs={"k": 16})

In [9]:
from langchain.prompts import PromptTemplate
template_det ="""
Based on the knowledge provided to you. You are expected to provide answer based on PICO framework.  You are a Professional medical scientist who have every knowledge in the medical field to answer user {context}. your answer should include description in detail (step by step, with each component of P,I,C and O) of each element about population (P), Intervention (I), Comparison (C) and Outcome (O) as stated in report/paper. What description should include is further guided. list each point separately.

Population (P) description: "Setting (including location and social context), Inclusion criteria, Exclusion criteria, Method of recruitment of participants (e.g. phone, mail, clinic patients), consent was taken or not, Total no. randomised (or total pop. at start of study for NRCTs),
Clusters (if applicable, no., type, no. people per cluster), Baseline imbalances, Withdrawals and exclusions (if not provided below by outcome), age, sex, race/ethnicity, Severity of illness,
Co-morbidities, Other relevant sociodemographics, Subgroups measured, Subgroups reported, and Notes."

Intervention (I) description: "Group name, No. randomised to group (specify whether no. people or clusters), Theoretical basis (include key references), Description (include sufficient detail for replication, e.g. content, dose, components), Duration of treatment period, Timing (e.g. frequency, duration of each episode),
Delivery (e.g. mechanism, medium, intensity, fidelity), Providers (e.g. no., profession, training, ethnicity etc. if relevant), Co-interventions, Economic information (i.e. intervention cost, changes in other costs as result of intervention), Resource requirements (e.g. staff numbers, cold chain, equipment), Integrity of delivery, Compliance."

Comparison (C) description: "Group name, No. randomised to group (specify whether no. people or clusters), Theoretical basis (include key references), Description (include sufficient detail for replication, e.g. content, dose, components), Duration of treatment period, Timing (e.g. frequency, duration of each episode),
Delivery (e.g. mechanism, medium, intensity, fidelity), Providers (e.g. no., profession, training, ethnicity etc. if relevant), Co-interventions, Economic information (i.e. intervention cost, changes in other costs as result of intervention),
Resource requirements (e.g. staff numbers, cold chain, equipment), Integrity of delivery, Compliance."

Outcome (O) description: "Outcome name, Time points measured (specify whether from start or end of intervention), Time points reported, Outcome definition (with diagnostic criteria if relevant), Person measuring/ reporting, Unit of measurement  (if relevant), Scales: upper and lower limits (indicate whether high  or low score is good), Is outcome/tool validated?,
Imputation of missing data (e.g. assumptions made for ITT analysis), Assumed risk estimate (e.g. baseline or population risk noted  in Background), Power (e.g. power & sample size calculation, level of power achieved)."

provide answer to the user (doctor, researcher or scientist) {context} which should be backed by evidance from PICO in a precise paragraph and tell what was the population, which intervention and comparisons were used and what was the outcome. ignore words like maybe or any other which may create ambiguity for users you answer should be solid based on evidance.
strictly complete your full answer within 150 words.


"""
Prompt=PromptTemplate(
    input_variables=["context"],
    template=template_det

)

In [11]:
from langchain.llms import OpenAI
from langchain_community.chat_models import ChatOpenAI
chat_3t = ChatOpenAI(model_name='gpt-3.5-turbo-1106', max_tokens=2048)
chat_4 = ChatOpenAI(model_name='gpt-4', max_tokens=2048)
chat_4t = ChatOpenAI(model_name='gpt-4-turbo', max_tokens=2048)
chat_4o = ChatOpenAI(model_name='gpt-4o', max_tokens=2048)
llm_openai=OpenAI(model_name='gpt-3.5-turbo')

/usr/local/lib/python3.10/dist-packages/langchain_community/llms/openai.py:253: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/llms/openai.py:1076: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


In [12]:
import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    return wrap_text_preserve_newlines(llm_response['result'])
    #print(wrap_text_preserve_newlines(llm_response['result']))
    #print('\nSources:')
    #for source in llm_response["source_documents"]:
        #print(source)

In [13]:
from langchain.chains import RetrievalQA

In [14]:
qa_4o_with_ins = RetrievalQA.from_chain_type (llm=chat_4o,
                                  chain_type="stuff",
                                  chain_type_kwargs={"prompt":Prompt},
                                  retriever=instruct_retriever,
                                  return_source_documents=False)

In [15]:
qa_4o_with_open = RetrievalQA.from_chain_type (llm=chat_4o,
                                  chain_type="stuff",
                                  chain_type_kwargs={"prompt":Prompt},
                                  retriever=open_retriever,
                                  return_source_documents=False)

In [16]:
qa_3t_with_ins = RetrievalQA.from_chain_type (llm=chat_3t,
                                  chain_type="stuff",
                                  chain_type_kwargs={"prompt":Prompt},
                                  retriever=instruct_retriever,
                                  return_source_documents=False)

In [17]:
qa_3t_with_open = RetrievalQA.from_chain_type (llm=chat_3t,
                                  chain_type="stuff",
                                  chain_type_kwargs={"prompt":Prompt},
                                  retriever=open_retriever,
                                  return_source_documents=False)

In [18]:
qa_4_with_ins = RetrievalQA.from_chain_type (llm=chat_4,
                                  chain_type="stuff",
                                  chain_type_kwargs={"prompt":Prompt},
                                  retriever=instruct_retriever,
                                  return_source_documents=False)

In [19]:
qa_4_with_open = RetrievalQA.from_chain_type (llm=chat_4,
                                  chain_type="stuff",
                                  chain_type_kwargs={"prompt":Prompt},
                                  retriever=open_retriever,
                                  return_source_documents=False)

In [20]:
qa_4t_with_ins = RetrievalQA.from_chain_type (llm=chat_4t,
                                  chain_type="stuff",
                                  chain_type_kwargs={"prompt":Prompt},
                                  retriever=instruct_retriever,
                                  return_source_documents=False)

In [21]:
qa_4t_with_open = RetrievalQA.from_chain_type (llm=chat_4t,
                                  chain_type="stuff",
                                  chain_type_kwargs={"prompt":Prompt},
                                  retriever=open_retriever,
                                  return_source_documents=False)

In [22]:
questions = ["In patients with type 2 diabetes, does probiotic ingestion decrease the incidence, quantity, and severity of GI adverse events associated with metformin treatment compared to metformin treatment alone?",
"In adults with type 2 diabetes who have not achieved adequate glycemic control with lifestyle changes, how does metformin compare to other antihyperglycemic drugs (such as sulfonylureas, insulin, or other antidiabetic medications) in reducing HbA1c levels, minimizing the risk of hypoglycemia and weight gain, and improving cardiovascular outcomes?",
"In youth aged 10-17 years with type 2 diabetes, what is the incidence of adverse events when treated with empagliflozin (10 mg or 25 mg) or linagliptin (5 mg) compared to placebo up to 26 weeks?",
"In individuals aged 16 years and older, what is the efficacy of the first dose of the BNT162b2 vaccine compared to a placebo in preventing COVID-19 infection 12 days post-administration?",
"In autologous hematopoietic stem cell transplantation (HSCT) recipients 18 years of age and older, how does the adjuvanted Recombinant Zoster Vaccine (RZV) compare to placebo in reducing the burden-of-illness (BOI) score for Herpes Zoster?",
]


ground_truths = ["probiotic ingestion can be beneficial in alleviating GI adverse events in patients with type 2 diabetes who are intolerant to metformin, highlighting the potential of probiotics as a supportive therapy in managing metformin intolerance in diabetic patients.",
"metformin is a recommended first-line treatment for type 2 diabetes due to its efficacy in reducing HbA1c levels, minimal risk of hypoglycemia, and potential cardiovascular benefits. However, individualized treatment considerations should be made based on patient factors, including comorbidities, preferences, and treatment goals.",
"Adverse events were reported in 64.2% of participants in the placebo group, 76.9% in the empagliflozin pooled doses group, and 71.2% in the linagliptin treatment group. Severe adverse events were reported in 3.8% of placebo-treated participants, 1.9% of empagliflozin pooled-treated participants, and 1.9% of linagliptin-treated participants. Hypoglycemia was the most frequently reported adverse event, with higher rates for those on active drug treatment compared to placebo. However, no severe hypoglycemia cases were reported. Overall, the rates of adverse events in non-responders re-randomized to empagliflozin 10 mg and 25 mg were low and comparable with rates in the placebo group",
"In individuals aged 16 years and older, the efficacy of the first dose of the BNT162b2 vaccine in preventing COVID-19 infection 12 days post-administration was noted to be 52% with a 95% confidence interval ranging from 29.5 to 68.4. This efficacy data was observed among participants who received the first dose of the BNT162b2 vaccine compared to those who received a placebo. The study findings indicate that even after the first dose, the BNT162b2 vaccine provides significant early protection against COVID-19, with a notable efficacy rate within a relatively short timeframe post-administration.",
"In autologous hematopoietic stem cell transplantation (HSCT) recipients 18 years of age and older, the adjuvanted Recombinant Zoster Vaccine (RZV) has been shown to significantly reduce the burden-of-illness (BOI) score for Herpes Zoster compared to placebo [T12]. This indicates that RZV is effective in reducing the impact of Herpes Zoster in this specific population, highlighting the importance of vaccination in HSCT recipients to prevent shingles-related complications.",
 ]


In [60]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    context_utilization,
)
def evaluate_RAG(questions,answers,contexts,ground_truths):
  from datasets import Dataset
  data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truths
  }

  dataset1 = Dataset.from_dict(data)
  result = evaluate(
    dataset = dataset1,
    #llm = llm_palm,
    #embeddings = instructor_embeddings,
    #is_async = True,
    raise_exceptions=False,
    metrics=[
        #context_utilization,
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
  )

  df = result.to_pandas()
  return df


In [48]:
answers = []
contexts = []
for query in questions:
  answers.append(process_llm_response(qa_4t_with_open(query)))
  contexts.append([docs.page_content for docs in open_retriever.get_relevant_documents(query)])

In [28]:
import nest_asyncio
nest_asyncio.apply()

In [58]:
results= evaluate_RAG(questions,answers,contexts,ground_truths)

In [41]:
results # 4t with open

,question,answer,contexts,ground_truth,context_precision,context_recall,faithfulness,answer_relevancy
0,"In patients with type 2 diabetes, does probiot...","In a randomized, double-blind, placebo-control...",[assessed tolerability of metformin (P <0.01/P...,probiotic ingestion can be beneficial in allev...,1.000000,0.5,1.000000,0.893958
1,In adults with type 2 diabetes who have not ac...,"Based on the PICO framework, the population co...",[cardiovascular complications and death. They ...,metformin is a recommended first-line treatmen...,1.000000,1.0,0.571429,0.849297
2,In youth aged 10-17 years with type 2 diabetes...,"In a multicenter, randomized, double-blind, pl...",[The objective of the study was to assess the ...,Adverse events were reported in 64.2% of parti...,1.000000,1.0,NaN,0.930323
3,"In individuals aged 16 years and older, what i...",In the BNT162b2 vaccine Phase 2/3 clinical tri...,[the BNT162b2 vaccine was assessed between dos...,"In individuals aged 16 years and older, the ef...",NaN,1.0,1.000000,0.874980
4,In autologous hematopoietic stem cell transpla...,Population (P): The study involved adult autol...,"[17. Serna L, Campora L, Chandrasekar P, et al...",In autologous hematopoietic stem cell transpla...,0.990136,1.0,1.000000,0.859718


In [50]:
results # 3t with open

,question,answer,contexts,ground_truth,context_precision,context_recall,faithfulness,answer_relevancy
0,"In patients with type 2 diabetes, does probiot...",Population: The study included adults with typ...,[assessed tolerability of metformin (P <0.01/P...,probiotic ingestion can be beneficial in allev...,1.000000,0.5,0.846154,0.895932
1,In adults with type 2 diabetes who have not ac...,"Based on the evidence from the PICO framework,...",[cardiovascular complications and death. They ...,metformin is a recommended first-line treatmen...,1.000000,NaN,0.833333,0.849297
2,In youth aged 10-17 years with type 2 diabetes...,Population (P): The study included participant...,[The objective of the study was to assess the ...,Adverse events were reported in 64.2% of parti...,NaN,1.0,NaN,0.785639
3,"In individuals aged 16 years and older, what i...",Population: The population studied included 37...,[the BNT162b2 vaccine was assessed between dos...,"In individuals aged 16 years and older, the ef...",1.000000,1.0,0.750000,0.844383
4,In autologous hematopoietic stem cell transpla...,Population: The study included autologous hema...,"[17. Serna L, Campora L, Chandrasekar P, et al...",In autologous hematopoietic stem cell transpla...,0.990136,0.5,1.000000,0.767655


In [47]:
results # 4 with open

,question,answer,contexts,ground_truth,context_precision,context_recall,faithfulness,answer_relevancy
0,"In patients with type 2 diabetes, does probiot...",The population (P) in the study were adults wi...,[assessed tolerability of metformin (P <0.01/P...,probiotic ingestion can be beneficial in allev...,1.0,0.5,1.000000,0.912098
1,In adults with type 2 diabetes who have not ac...,In the population of patients with type 2 diab...,[cardiovascular complications and death. They ...,metformin is a recommended first-line treatmen...,1.0,NaN,1.000000,0.876725
2,In youth aged 10-17 years with type 2 diabetes...,The study assessed the efficacy and safety of ...,[The objective of the study was to assess the ...,Adverse events were reported in 64.2% of parti...,1.0,1.0,0.857143,0.926593
3,"In individuals aged 16 years and older, what i...","Based on the PICO framework, the Population (P...",[the BNT162b2 vaccine was assessed between dos...,"In individuals aged 16 years and older, the ef...",NaN,1.0,1.000000,0.890008
4,In autologous hematopoietic stem cell transpla...,The PICO framework for the Herpes Zoster (HZ) ...,"[17. Serna L, Campora L, Chandrasekar P, et al...",In autologous hematopoietic stem cell transpla...,NaN,0.5,NaN,0.846746


In [44]:
results # 4o with open

,question,answer,contexts,ground_truth,context_precision,context_recall,faithfulness,answer_relevancy
0,"In patients with type 2 diabetes, does probiot...","In a prospective, 32-week, randomized, cross-o...",[assessed tolerability of metformin (P <0.01/P...,probiotic ingestion can be beneficial in allev...,1.000000,0.5,0.714286,0.879860
1,In adults with type 2 diabetes who have not ac...,**Population (P):** Adults with type 2 diabete...,[cardiovascular complications and death. They ...,metformin is a recommended first-line treatmen...,NaN,NaN,NaN,0.849297
2,In youth aged 10-17 years with type 2 diabetes...,"In this double-blind, placebo-controlled trial...",[The objective of the study was to assess the ...,Adverse events were reported in 64.2% of parti...,1.000000,1.0,0.400000,0.921468
3,"In individuals aged 16 years and older, what i...","Based on the provided data, the phase 2/3 clin...",[the BNT162b2 vaccine was assessed between dos...,"In individuals aged 16 years and older, the ef...",1.000000,1.0,1.000000,0.878336
4,In autologous hematopoietic stem cell transpla...,The study evaluated the efficacy and safety of...,"[17. Serna L, Campora L, Chandrasekar P, et al...",In autologous hematopoietic stem cell transpla...,0.990136,0.5,0.785714,0.864308


In [37]:
results # 4t with instructor

,question,answer,contexts,ground_truth,context_precision,context_recall,faithfulness,answer_relevancy
0,"In patients with type 2 diabetes, does probiot...",In a randomized controlled trial targeting pat...,"[the host” [24]. In this trial, our primary ai...",probiotic ingestion can be beneficial in allev...,1.000000,0.50,0.916667,0.871313
1,In adults with type 2 diabetes who have not ac...,"In the context of managing type 2 diabetes, pa...",[cardiovascular complications and death. They ...,metformin is a recommended first-line treatmen...,0.926840,0.75,0.909091,0.849297
2,In youth aged 10-17 years with type 2 diabetes...,"In this double-blind, placebo-controlled trial...",[The objective of the study was to assess the ...,Adverse events were reported in 64.2% of parti...,NaN,0.80,1.000000,0.920327
3,"In individuals aged 16 years and older, what i...","The BNT162b2 mRNA COVID-19 vaccine, developed ...",[of the acclaimed BNT162b2 vaccine in December...,"In individuals aged 16 years and older, the ef...",0.963216,1.00,0.000000,0.924226
4,In autologous hematopoietic stem cell transpla...,"In the randomized, placebo-controlled, multice...","[14. Curran D, Szer J, Matthews S, et al. Reco...",In autologous hematopoietic stem cell transpla...,0.886393,0.50,1.000000,0.838941


In [34]:
results # 4 with instructor

,question,answer,contexts,ground_truth,context_precision,context_recall,faithfulness,answer_relevancy
0,"In patients with type 2 diabetes, does probiot...",The study included patients with Type 2 Diabet...,"[the host” [24]. In this trial, our primary ai...",probiotic ingestion can be beneficial in allev...,1.000000,0.50,0.857143,0.918066
1,In adults with type 2 diabetes who have not ac...,The PICO framework applied to the provided inf...,[cardiovascular complications and death. They ...,metformin is a recommended first-line treatmen...,0.926840,0.75,1.000000,0.849297
2,In youth aged 10-17 years with type 2 diabetes...,The study aimed to evaluate the effect of empa...,[The objective of the study was to assess the ...,Adverse events were reported in 64.2% of parti...,NaN,1.00,1.000000,0.874026
3,"In individuals aged 16 years and older, what i...","The population (P) in question involved 43,448...",[of the acclaimed BNT162b2 vaccine in December...,"In individuals aged 16 years and older, the ef...",1.000000,1.00,1.000000,0.887585
4,In autologous hematopoietic stem cell transpla...,The PICO framework for this study is as follow...,"[14. Curran D, Szer J, Matthews S, et al. Reco...",In autologous hematopoietic stem cell transpla...,0.886393,1.00,NaN,0.753559


In [31]:
results # 3t with instructor

,question,answer,contexts,ground_truth,context_precision,context_recall,faithfulness,answer_relevancy
0,"In patients with type 2 diabetes, does probiot...",Population: The population of interest in this...,"[the host” [24]. In this trial, our primary ai...",probiotic ingestion can be beneficial in allev...,1.000000,0.50,0.75,0.833484
1,In adults with type 2 diabetes who have not ac...,"Based on the evidence from the study, the popu...",[cardiovascular complications and death. They ...,metformin is a recommended first-line treatmen...,0.867039,0.75,1.00,0.849297
2,In youth aged 10-17 years with type 2 diabetes...,Population: The study population consisted of ...,[The objective of the study was to assess the ...,Adverse events were reported in 64.2% of parti...,NaN,1.00,1.00,0.785639
3,"In individuals aged 16 years and older, what i...",The BNT162b2 vaccine has shown significant eff...,[of the acclaimed BNT162b2 vaccine in December...,"In individuals aged 16 years and older, the ef...",1.000000,1.00,NaN,0.911076
4,In autologous hematopoietic stem cell transpla...,Population: The population of interest in thes...,"[14. Curran D, Szer J, Matthews S, et al. Reco...",In autologous hematopoietic stem cell transpla...,0.960480,0.50,1.00,0.765528


In [27]:
results # 4o with instructor

,question,answer,contexts,ground_truth,context_precision,context_recall,faithfulness,answer_relevancy
0,"In patients with type 2 diabetes, does probiot...","In this trial, the population (P) consisted of...","[the host” [24]. In this trial, our primary ai...",probiotic ingestion can be beneficial in allev...,1.000000,0.50,NaN,0.798605
1,In adults with type 2 diabetes who have not ac...,Based on the provided references and the PICO ...,[cardiovascular complications and death. They ...,metformin is a recommended first-line treatmen...,0.867039,0.75,0.444444,0.849297
2,In youth aged 10-17 years with type 2 diabetes...,The study objective was to assess the efficacy...,[The objective of the study was to assess the ...,Adverse events were reported in 64.2% of parti...,NaN,0.80,1.000000,0.915270
3,"In individuals aged 16 years and older, what i...","The BNT162b2 vaccine, developed by Pfizer and ...",[of the acclaimed BNT162b2 vaccine in December...,"In individuals aged 16 years and older, the ef...",0.963216,1.00,NaN,0.889351
4,In autologous hematopoietic stem cell transpla...,In the randomized placebo-controlled ZOE-HSCT ...,"[14. Curran D, Szer J, Matthews S, et al. Reco...",In autologous hematopoietic stem cell transpla...,0.957659,0.50,0.857143,0.840391
